<a href="https://colab.research.google.com/github/wsokal/gold_buying_app/blob/main/aipokertrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install treys


In [2]:
import random
import sqlite3
from treys import Deck, Evaluator, Card

# --- SETUP: SQLite Database for Recording Statistics ---

# Connect to (or create) a local SQLite database file.
conn = sqlite3.connect('texas_holdem_stats.db')
cursor = conn.cursor()

# Create a table to store the starting hand statistics.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS starter_hand_stats (
        hand_category TEXT PRIMARY KEY,
        wins REAL,
        total INTEGER
    )
''')
conn.commit()


# --- HELPER FUNCTIONS ---

def classify_starter_hand(card1, card2):
    """
    Convert two hole cards into one of 169 starting hand categories.
    Uses card rank (e.g. 'A', 'K', 'Q', etc.) and suitedness.
    For a pair, returns e.g., "AA".
    For two different cards, returns e.g., "AKs" for suited or "AKo" for offsuit.
    """
    # Convert card integers into their string representation, e.g., "Ah", "Kd", etc.
    card_str1 = Card.int_to_str(card1)  # e.g., "Ah"
    card_str2 = Card.int_to_str(card2)  # e.g., "Kd"

    rank1, suit1 = card_str1[0], card_str1[1]
    rank2, suit2 = card_str2[0], card_str2[1]

    # Define the rank order for comparison.
    ranks_order = "23456789TJQKA"

    # Order the ranks so that the highest appears first.
    if ranks_order.index(rank1) < ranks_order.index(rank2):
        rank1, rank2 = rank2, rank1
        suit1, suit2 = suit2, suit1

    if rank1 == rank2:
        # Pocket pair: no suitedness marker.
        return rank1 + rank2
    else:
        # Non-paired hands: add 's' if suited, otherwise 'o'
        suited = 's' if suit1 == suit2 else 'o'
        return rank1 + rank2 + suited


def update_database(starting_hands, winners):
    """
    For each player’s starting hand category, update the running win/total counts.
    If multiple players tie for the win, split the win credit equally.
    """
    for i, hand_category in enumerate(starting_hands):
        # Determine win fraction: if the player is in the list of winners, they get a share of 1.
        win_fraction = 1.0 / len(winners) if i in winners else 0.0

        # Insert or update the row in the table.
        cursor.execute('''
            INSERT INTO starter_hand_stats (hand_category, wins, total)
            VALUES (?, ?, ?)
            ON CONFLICT(hand_category) DO UPDATE SET
                wins = wins + ?,
                total = total + ?
        ''', (hand_category, win_fraction, 1, win_fraction, 1))
    conn.commit()


# --- SIMULATION FUNCTIONS ---

def simulate_game():
    """
    Simulate a single Texas Hold'em game with 10 players.
    - Every player is dealt 2 hole cards.
    - 5 community cards are dealt.
    - All players stay in (no folding).
    - The best hand wins (ties are split).

    Returns:
        starting_hands: A list (length 10) of starting hand categories (e.g., "AKs", "77", etc.)
        winners: A list of player indices (0-based) that won the game.
    """
    deck = Deck()
    evaluator = Evaluator()

    # Deal hole cards to 10 players.
    players_hole = [deck.draw(2) for _ in range(10)]

    # Deal 5 community cards.
    board = deck.draw(5)

    # Evaluate each player's best hand using the evaluator.
    scores = [evaluator.evaluate(hand, board) for hand in players_hole]
    best_score = min(scores)  # Lower score means a stronger hand.
    winners = [i for i, score in enumerate(scores) if score == best_score]

    # Classify each player's starting hand.
    starting_hands = [classify_starter_hand(hand[0], hand[1]) for hand in players_hole]

    return starting_hands, winners


def run_simulations(num_iterations=10000):
    """
    Run a number of simulated games and update the database with results.
    """
    for i in range(1, num_iterations + 1):
        starting_hands, winners = simulate_game()
        update_database(starting_hands, winners)

        if i % 1000 == 0:
            print(f"Completed {i} iterations")


# --- MAIN ---

if __name__ == "__main__":
    total_iterations = 10000  # Change this value as needed.
    run_simulations(total_iterations)

    # Query and print the results.
    cursor.execute("SELECT * FROM starter_hand_stats ORDER BY total DESC")
    results = cursor.fetchall()
    print("\n=== Starter Hand Statistics ===")
    for hand_category, wins, total in results:
        win_rate = wins / total if total > 0 else 0
        print(f"Hand {hand_category}: Wins = {wins:.3f}, Total = {total}, Win Rate = {win_rate:.3f}")

    # Close the database connection.
    conn.close()


Completed 1000 iterations
Completed 2000 iterations
Completed 3000 iterations
Completed 4000 iterations
Completed 5000 iterations
Completed 6000 iterations
Completed 7000 iterations
Completed 8000 iterations
Completed 9000 iterations
Completed 10000 iterations

=== Starter Hand Statistics ===
Hand 43o: Wins = 65.283, Total = 983, Win Rate = 0.066
Hand T7o: Wins = 95.833, Total = 969, Win Rate = 0.099
Hand Q2o: Wins = 62.333, Total = 957, Win Rate = 0.065
Hand T3o: Wins = 60.667, Total = 957, Win Rate = 0.063
Hand J7o: Wins = 85.783, Total = 957, Win Rate = 0.090
Hand 82o: Wins = 44.433, Total = 955, Win Rate = 0.047
Hand J6o: Wins = 61.417, Total = 953, Win Rate = 0.064
Hand Q4o: Wins = 71.517, Total = 951, Win Rate = 0.075
Hand A9o: Wins = 108.183, Total = 950, Win Rate = 0.114
Hand T8o: Wins = 96.283, Total = 950, Win Rate = 0.101
Hand A4o: Wins = 109.500, Total = 947, Win Rate = 0.116
Hand 97o: Wins = 86.850, Total = 947, Win Rate = 0.092
Hand JTo: Wins = 128.683, Total = 944, Win R

In [3]:
import random

# Define lists/dictionaries of variables that influence scenario generation.
POSITIONS = ['Under the Gun', 'Middle Position', 'Hijack', 'Cutoff', 'Button', 'Small Blind', 'Big Blind']
CHIP_STACKS = ['Short', 'Medium', 'Deep']  # Relative to tournament average; could be more numeric too.
PLAYER_TILTS = ['Calm', 'Slightly Tilted', 'On Tilt']
EXPERIENCE_LEVELS = ['High', 'Medium', 'Low']
TABLE_DYNAMICS = ['Aggressive', 'Passive', 'Mixed']
BLUFFING_CUES = ['No clear bluff', 'Subtle tell of bluffing', 'Strong indication of bluffing']

# Additional scenario aspects might include tournament stage, pot size, and opponent ranges.
TOURNAMENT_STAGES = ['Early', 'Middle', 'Late', 'Bubble', 'Final Table']
POT_SIZES = ['Small', 'Moderate', 'Large']

# Decision actions available for most scenarios.
ACTIONS = ['Check', 'Raise', 'Fold']
# Some scenarios might also offer additional actions.
ADDITIONAL_ACTIONS = ['Call', 'Re-raise', 'All-in']

def generate_scenario():
    """
    Randomly generates a tournament Texas Hold’em scenario.
    Returns a dictionary containing the scenario details.
    """
    scenario = {
        'tournament_stage': random.choice(TOURNAMENT_STAGES),
        'position': random.choice(POSITIONS),
        'chip_stack': random.choice(CHIP_STACKS),
        'table_dynamics': random.choice(TABLE_DYNAMICS),
        'pot_size': random.choice(POT_SIZES),
        'opponent_tilt': random.choice(PLAYER_TILTS),
        'experience_level': random.choice(EXPERIENCE_LEVELS),
        'bluffing_cue': random.choice(BLUFFING_CUES)
    }

    # Optionally, decide if this scenario requires an extra decision.
    scenario['requires_extra_decision'] = random.choice([True, False])

    return scenario

def describe_scenario(scenario):
    """
    Returns a multi-line string describing the scenario.
    """
    description = (
        f"Tournament Stage: {scenario['tournament_stage']}\n"
        f"Your Position: {scenario['position']}\n"
        f"Chip Stack: {scenario['chip_stack']}\n"
        f"Pot Size: {scenario['pot_size']}\n"
        f"Table Dynamics: {scenario['table_dynamics']}\n"
        f"Opponent Tilt Level: {scenario['opponent_tilt']}\n"
        f"Your Experience Level: {scenario['experience_level']}\n"
        f"Observation: {scenario['bluffing_cue']}\n"
    )
    if scenario['requires_extra_decision']:
        description += "This situation appears to be more nuanced—additional decision options may apply.\n"
    return description

def get_user_decision(scenario):
    """
    Prompts the user for a decision on the scenario.
    Returns the user decision.
    """
    # Base options are always Check, Raise, or Fold.
    options = ACTIONS.copy()
    if scenario['requires_extra_decision']:
        # If additional decisions are warranted, add them.
        options.extend(ADDITIONAL_ACTIONS)

    print("Scenario:")
    print("-------------------")
    print(describe_scenario(scenario))
    print("What is your decision? Options:", ", ".join(options))

    decision = input("Enter your decision: ").strip()
    while decision not in options:
        print("Invalid input. Please choose from:", ", ".join(options))
        decision = input("Enter your decision: ").strip()

    return decision

def run_scenario_generator(num_scenarios=5):
    """
    Runs a series of scenarios for user input.
    In a complete trainer system, user responses might be logged or analyzed.
    """
    responses = []
    for i in range(num_scenarios):
        print(f"\n=== Scenario {i+1} ===")
        scenario = generate_scenario()
        decision = get_user_decision(scenario)
        responses.append({
            'scenario': scenario,
            'decision': decision
        })
        print(f"You decided to: {decision}\n")

    # For now, we simply return the responses. In a real system, you'd record these in a database.
    return responses

if __name__ == "__main__":
    print("Welcome to the Poker AI Pro Trainer Scenario Generator!")
    num = input("How many scenarios would you like to run? (default 5): ").strip()
    try:
        num_scenarios = int(num) if num else 5
    except ValueError:
        num_scenarios = 5

    responses = run_scenario_generator(num_scenarios)

    # Example: Output responses. In a full system, you might analyze or store these.
    print("\n=== Summary of Responses ===")
    for idx, entry in enumerate(responses, 1):
        scenario = entry['scenario']
        decision = entry['decision']
        print(f"Scenario {idx}: Position = {scenario['position']}, "
              f"Chip Stack = {scenario['chip_stack']}, "
              f"Action Taken = {decision}")


Welcome to the Poker AI Pro Trainer Scenario Generator!
How many scenarios would you like to run? (default 5): 10

=== Scenario 1 ===
Scenario:
-------------------
Tournament Stage: Final Table
Your Position: Cutoff
Chip Stack: Short
Pot Size: Large
Table Dynamics: Passive
Opponent Tilt Level: On Tilt
Your Experience Level: High
Observation: Strong indication of bluffing

What is your decision? Options: Check, Raise, Fold
Enter your decision: raise
Invalid input. Please choose from: Check, Raise, Fold
Enter your decision: Raise
You decided to: Raise


=== Scenario 2 ===
Scenario:
-------------------
Tournament Stage: Bubble
Your Position: Hijack
Chip Stack: Medium
Pot Size: Small
Table Dynamics: Passive
Opponent Tilt Level: Calm
Your Experience Level: Medium
Observation: Strong indication of bluffing
This situation appears to be more nuanced—additional decision options may apply.

What is your decision? Options: Check, Raise, Fold, Call, Re-raise, All-in
Enter your decision: All-in
You

KeyboardInterrupt: Interrupted by user

In [4]:
import random

# ---------------------------
# Helper Functions and Setup
# ---------------------------

# Define the ranks and suits for a deck of cards.
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
SUITS = ['♠', '♥', '♦', '♣']

def generate_deck():
    """Creates and shuffles a deck of 52 cards represented as strings (e.g., 'A♠')."""
    deck = [rank + suit for rank in RANKS for suit in SUITS]
    random.shuffle(deck)
    return deck

def draw_cards(deck, count):
    """Draws 'count' cards from the deck."""
    return [deck.pop() for _ in range(count)]

# ---------------------------
# Scenario Generation
# ---------------------------

def generate_scenario():
    """
    Generates a detailed tournament scenario that includes:
      - Your hole cards.
      - Current street (Preflop, Flop, Turn, River) and board cards.
      - Tournament stage and pot size.
      - Number of players remaining in the hand.
      - Your position and chip stack.
      - A series of opponent actions before it's your turn.
      - A list of available action options.
    """
    deck = generate_deck()

    # 1. Your Hole Cards
    hole_cards = draw_cards(deck, 2)

    # 2. Determine the current street and generate board cards if applicable.
    street = random.choice(['Preflop', 'Flop', 'Turn', 'River'])
    board = []
    if street == 'Flop':
        board = draw_cards(deck, 3)
    elif street == 'Turn':
        board = draw_cards(deck, 3)
        board.append(draw_cards(deck, 1)[0])
    elif street == 'River':
        board = draw_cards(deck, 3)
        board.append(draw_cards(deck, 1)[0])
        board.append(draw_cards(deck, 1)[0])

    # 3. Tournament and Table Details
    tournament_stages = ['Early', 'Middle', 'Late', 'Bubble', 'Final Table']
    tournament_stage = random.choice(tournament_stages)
    pot_size = random.choice(['Small', 'Moderate', 'Large'])

    # 4. Determine the number of players in the hand (including you)
    players_in_hand = random.randint(2, 10)

    # 5. Your Position and Chip Stack
    positions = ['Under the Gun', 'UTG+1', 'Middle Position', 'Hijack', 'Cutoff', 'Button', 'Small Blind', 'Big Blind']
    your_position = random.choice(positions)
    chip_stacks = ['Short', 'Medium', 'Deep']
    your_chip_stack = random.choice(chip_stacks)

    # 6. Opponent Actions
    # Simulate a series of actions by opponents who acted before you.
    # For example, "Player 1: Raise 3x", "Player 2: All-in", etc.
    num_opponents_acting = random.randint(1, players_in_hand - 1)
    possible_actions = [
        "Fold",
        "Call",
        "Raise 3x Blind",
        "Raise 4x Blind",
        "All-in"
    ]
    opponent_actions = [random.choice(possible_actions) for _ in range(num_opponents_acting)]

    # 7. Determine available options for your action.
    # Base options:
    options = ["Check", "Call", "Raise", "Fold"]
    # If any opponent went all-in, add an extra option.
    if any(action == "All-in" for action in opponent_actions):
        options.append("Re-raise All-in")

    scenario = {
        'tournament_stage': tournament_stage,
        'street': street,
        'your_hole_cards': hole_cards,
        'board': board,
        'players_in_hand': players_in_hand,
        'your_position': your_position,
        'your_chip_stack': your_chip_stack,
        'pot_size': pot_size,
        'opponent_actions': opponent_actions,
        'available_options': options
    }
    return scenario

def describe_scenario(scenario):
    """
    Returns a detailed multi-line string that describes the scenario,
    including your hand, board state, tournament context, table dynamics,
    and the betting action that occurred before your turn.
    """
    description = "\n--- Poker Tournament Scenario ---\n"
    description += f"Tournament Stage: {scenario['tournament_stage']}\n"
    description += f"Current Street: {scenario['street']}\n"
    description += f"Your Hole Cards: {', '.join(scenario['your_hole_cards'])}\n"
    if scenario['street'] != 'Preflop':
        description += f"Board Cards: {', '.join(scenario['board'])}\n"
    description += f"Players in Hand (including you): {scenario['players_in_hand']}\n"
    description += f"Your Position: {scenario['your_position']}\n"
    description += f"Your Chip Stack: {scenario['your_chip_stack']}\n"
    description += f"Pot Size: {scenario['pot_size']}\n"
    description += "\nActions before you:\n"
    for idx, action in enumerate(scenario['opponent_actions'], 1):
        description += f"  Opponent {idx}: {action}\n"
    description += "\nAvailable Options for Your Action: " + ", ".join(scenario['available_options']) + "\n"
    description += "What is your decision?\n"
    return description

def get_user_decision(scenario):
    """
    Displays the scenario, prompts the user for a decision, and validates the input.
    Returns the user's decision.
    """
    print(describe_scenario(scenario))
    decision = input("Enter your decision: ").strip()
    while decision not in scenario['available_options']:
        print("Invalid choice. Please choose from:", ", ".join(scenario['available_options']))
        decision = input("Enter your decision: ").strip()
    return decision

def run_scenario_generator(num_scenarios=5):
    """
    Generates a specified number of scenarios, collects user responses, and returns them.
    """
    responses = []
    for i in range(num_scenarios):
        print(f"\n=== Scenario {i+1} ===")
        scenario = generate_scenario()
        decision = get_user_decision(scenario)
        responses.append({
            'scenario': scenario,
            'decision': decision
        })
        print(f"\nYou decided to: {decision}\n")
    return responses

# ---------------------------
# Main Program
# ---------------------------

if __name__ == '__main__':
    print("Welcome to the Poker AI Pro Trainer Scenario Generator!")
    try:
        num = int(input("How many scenarios would you like to generate? (default 5): ").strip() or 5)
    except ValueError:
        num = 5
    responses = run_scenario_generator(num)

    # Optionally, print a summary of responses.
    print("\n=== Summary of Your Decisions ===")
    for idx, entry in enumerate(responses, 1):
        scenario = entry['scenario']
        decision = entry['decision']
        print(f"Scenario {idx}:")
        print(f"  Position: {scenario['your_position']} | Chip Stack: {scenario['your_chip_stack']} | Street: {scenario['street']}")
        print(f"  Your Cards: {', '.join(scenario['your_hole_cards'])} | Board: {', '.join(scenario['board']) if scenario['board'] else 'N/A'}")
        print(f"  Opponents' Actions: {', '.join(scenario['opponent_actions'])}")
        print(f"  Your Decision: {decision}\n")


Welcome to the Poker AI Pro Trainer Scenario Generator!
How many scenarios would you like to generate? (default 5): 5

=== Scenario 1 ===

--- Poker Tournament Scenario ---
Tournament Stage: Early
Current Street: Turn
Your Hole Cards: 5♠, 9♠
Board Cards: 7♠, 9♥, 5♣, J♥
Players in Hand (including you): 2
Your Position: UTG+1
Your Chip Stack: Short
Pot Size: Large

Actions before you:
  Opponent 1: Call

Available Options for Your Action: Check, Call, Raise, Fold
What is your decision?

Enter your decision: Raise

You decided to: Raise


=== Scenario 2 ===

--- Poker Tournament Scenario ---
Tournament Stage: Late
Current Street: Flop
Your Hole Cards: A♠, 5♣
Board Cards: T♣, 6♥, 9♣
Players in Hand (including you): 6
Your Position: Under the Gun
Your Chip Stack: Short
Pot Size: Large

Actions before you:
  Opponent 1: All-in

Available Options for Your Action: Check, Call, Raise, Fold, Re-raise All-in
What is your decision?



KeyboardInterrupt: Interrupted by user

In [5]:
import random

# ---------------------------
# Helper Functions and Setup
# ---------------------------

# Define the ranks and suits for a deck of cards.
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
SUITS = ['♠', '♥', '♦', '♣']

def generate_deck():
    """Creates and shuffles a deck of 52 cards represented as strings (e.g., 'A♠')."""
    deck = [rank + suit for rank in RANKS for suit in SUITS]
    random.shuffle(deck)
    return deck

def draw_cards(deck, count):
    """Draws 'count' cards from the deck."""
    return [deck.pop() for _ in range(count)]

# ---------------------------
# Scenario Generation
# ---------------------------

def generate_scenario():
    """
    Generates a detailed tournament scenario that includes:
      - Your hole cards.
      - Current street (Preflop, Flop, Turn, River) and board cards.
      - Tournament stage and pot size.
      - Number of players remaining in the hand.
      - Your position and chip stack.
      - A series of opponent actions before it's your turn.
      - A list of available action options.
    """
    deck = generate_deck()

    # 1. Your Hole Cards
    hole_cards = draw_cards(deck, 2)

    # 2. Determine the current street and generate board cards if applicable.
    street = random.choice(['Preflop', 'Flop', 'Turn', 'River'])
    board = []
    if street == 'Flop':
        board = draw_cards(deck, 3)
    elif street == 'Turn':
        board = draw_cards(deck, 3)
        board.append(draw_cards(deck, 1)[0])
    elif street == 'River':
        board = draw_cards(deck, 3)
        board.append(draw_cards(deck, 1)[0])
        board.append(draw_cards(deck, 1)[0])

    # 3. Tournament and Table Details
    tournament_stages = ['Early', 'Middle', 'Late', 'Bubble', 'Final Table']
    tournament_stage = random.choice(tournament_stages)
    pot_size = random.choice(['Small', 'Moderate', 'Large'])

    # 4. Determine the number of players in the hand (including you)
    players_in_hand = random.randint(2, 10)

    # 5. Your Position and Chip Stack
    positions = ['Under the Gun', 'UTG+1', 'Middle Position', 'Hijack', 'Cutoff', 'Button', 'Small Blind', 'Big Blind']
    your_position = random.choice(positions)
    chip_stacks = ['Short', 'Medium', 'Deep']
    your_chip_stack = random.choice(chip_stacks)

    # 6. Opponent Actions
    # Simulate a series of actions by opponents who acted before you.
    # For example, "Player 1: Raise 3x", "Player 2: All-in", etc.
    num_opponents_acting = random.randint(1, players_in_hand - 1)
    possible_actions = [
        "Fold",
        "Call",
        "Raise 3x Blind",
        "Raise 4x Blind",
        "All-in"
    ]
    opponent_actions = [random.choice(possible_actions) for _ in range(num_opponents_acting)]

    # 7. Determine available options for your action.
    # Base options:
    options = ["Check", "Call", "Raise", "Fold"]
    # If any opponent went all-in, add an extra option.
    if any(action == "All-in" for action in opponent_actions):
        options.append("Re-raise All-in")

    scenario = {
        'tournament_stage': tournament_stage,
        'street': street,
        'your_hole_cards': hole_cards,
        'board': board,
        'players_in_hand': players_in_hand,
        'your_position': your_position,
        'your_chip_stack': your_chip_stack,
        'pot_size': pot_size,
        'opponent_actions': opponent_actions,
        'available_options': options
    }
    return scenario

def describe_scenario(scenario):
    """
    Returns a detailed multi-line string that describes the scenario,
    including your hand, board state, tournament context, table dynamics,
    and the betting action that occurred before your turn.
    """
    description = "\n--- Poker Tournament Scenario ---\n"
    description += f"Tournament Stage: {scenario['tournament_stage']}\n"
    description += f"Current Street: {scenario['street']}\n"
    description += f"Your Hole Cards: {', '.join(scenario['your_hole_cards'])}\n"
    if scenario['street'] != 'Preflop':
        description += f"Board Cards: {', '.join(scenario['board'])}\n"
    description += f"Players in Hand (including you): {scenario['players_in_hand']}\n"
    description += f"Your Position: {scenario['your_position']}\n"
    description += f"Your Chip Stack: {scenario['your_chip_stack']}\n"
    description += f"Pot Size: {scenario['pot_size']}\n"
    description += "\nActions before you:\n"
    for idx, action in enumerate(scenario['opponent_actions'], 1):
        description += f"  Opponent {idx}: {action}\n"
    description += "\nAvailable Options for Your Action: " + ", ".join(scenario['available_options']) + "\n"
    description += "What is your decision?\n"
    return description

def get_user_decision(scenario):
    """
    Displays the scenario, prompts the user for a decision, and validates the input.
    Returns the user's decision.
    """
    print(describe_scenario(scenario))
    decision = input("Enter your decision: ").strip()
    while decision not in scenario['available_options']:
        print("Invalid choice. Please choose from:", ", ".join(scenario['available_options']))
        decision = input("Enter your decision: ").strip()
    return decision

def run_scenario_generator(num_scenarios=5):
    """
    Generates a specified number of scenarios, collects user responses, and returns them.
    """
    responses = []
    for i in range(num_scenarios):
        print(f"\n=== Scenario {i+1} ===")
        scenario = generate_scenario()
        decision = get_user_decision(scenario)
        responses.append({
            'scenario': scenario,
            'decision': decision
        })
        print(f"\nYou decided to: {decision}\n")
    return responses

# ---------------------------
# Main Program
# ---------------------------

if __name__ == '__main__':
    print("Welcome to the Poker AI Pro Trainer Scenario Generator!")
    try:
        num = int(input("How many scenarios would you like to generate? (default 5): ").strip() or 5)
    except ValueError:
        num = 5
    responses = run_scenario_generator(num)

    # Optionally, print a summary of responses.
    print("\n=== Summary of Your Decisions ===")
    for idx, entry in enumerate(responses, 1):
        scenario = entry['scenario']
        decision = entry['decision']
        print(f"Scenario {idx}:")
        print(f"  Position: {scenario['your_position']} | Chip Stack: {scenario['your_chip_stack']} | Street: {scenario['street']}")
        print(f"  Your Cards: {', '.join(scenario['your_hole_cards'])} | Board: {', '.join(scenario['board']) if scenario['board'] else 'N/A'}")
        print(f"  Opponents' Actions: {', '.join(scenario['opponent_actions'])}")
        print(f"  Your Decision: {decision}\n")


Welcome to the Poker AI Pro Trainer Scenario Generator!
How many scenarios would you like to generate? (default 5): 5

=== Scenario 1 ===

--- Poker Tournament Scenario ---
Tournament Stage: Final Table
Current Street: Flop
Your Hole Cards: J♣, 7♦
Board Cards: 2♦, K♥, K♣
Players in Hand (including you): 3
Your Position: Cutoff
Your Chip Stack: Short
Pot Size: Moderate

Actions before you:
  Opponent 1: Raise 3x Blind

Available Options for Your Action: Check, Call, Raise, Fold
What is your decision?



KeyboardInterrupt: Interrupted by user

In [6]:
import random

# ---------------------------
# Helper Functions and Setup
# ---------------------------

RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
SUITS = ['♠', '♥', '♦', '♣']

def generate_deck():
    """Creates and shuffles a deck of 52 cards represented as strings (e.g., 'A♠')."""
    deck = [rank + suit for rank in RANKS for suit in SUITS]
    random.shuffle(deck)
    return deck

def draw_cards(deck, count):
    """Draws 'count' cards from the deck."""
    return [deck.pop() for _ in range(count)]

# ---------------------------
# Scenario Generation
# ---------------------------

def generate_scenario():
    """
    Generates a detailed tournament scenario that includes:
      - Your hole cards.
      - Current street (Preflop, Flop, Turn, River) and board cards.
      - Tournament stage and pot size.
      - Number of players remaining in the hand.
      - Your position and chip stack.
      - A series of opponent actions before it's your turn.
      - A list of available action options.
    Returns the scenario as a dictionary.
    """
    deck = generate_deck()

    # 1. Your Hole Cards
    hole_cards = draw_cards(deck, 2)

    # 2. Determine the current street and generate board cards if applicable.
    street = random.choice(['Preflop', 'Flop', 'Turn', 'River'])
    board = []
    if street == 'Flop':
        board = draw_cards(deck, 3)
    elif street == 'Turn':
        board = draw_cards(deck, 3)
        board.append(draw_cards(deck, 1)[0])
    elif street == 'River':
        board = draw_cards(deck, 3)
        board.append(draw_cards(deck, 1)[0])
        board.append(draw_cards(deck, 1)[0])

    # 3. Tournament and Table Details
    tournament_stages = ['Early', 'Middle', 'Late', 'Bubble', 'Final Table']
    tournament_stage = random.choice(tournament_stages)
    pot_size = random.choice(['Small', 'Moderate', 'Large'])

    # 4. Determine the number of players in the hand (including you)
    players_in_hand = random.randint(2, 10)

    # 5. Your Position and Chip Stack
    positions = ['Under the Gun', 'UTG+1', 'Middle Position', 'Hijack', 'Cutoff', 'Button', 'Small Blind', 'Big Blind']
    your_position = random.choice(positions)
    chip_stacks = ['Short', 'Medium', 'Deep']
    your_chip_stack = random.choice(chip_stacks)

    # 6. Opponent Actions
    num_opponents_acting = random.randint(1, players_in_hand - 1)
    possible_actions = [
        "Fold",
        "Call",
        "Raise 3x Blind",
        "Raise 4x Blind",
        "All-in"
    ]
    opponent_actions = [random.choice(possible_actions) for _ in range(num_opponents_acting)]

    # 7. Determine available options for your action.
    options = ["Check", "Call", "Raise", "Fold"]
    if any(action == "All-in" for action in opponent_actions):
        options.append("Re-raise All-in")

    scenario = {
        'tournament_stage': tournament_stage,
        'street': street,
        'your_hole_cards': hole_cards,
        'board': board,
        'players_in_hand': players_in_hand,
        'your_position': your_position,
        'your_chip_stack': your_chip_stack,
        'pot_size': pot_size,
        'opponent_actions': opponent_actions,
        'available_options': options
    }
    return scenario

def describe_scenario(scenario):
    """
    Returns a detailed multi-line string that describes the scenario,
    including your hand, board state, tournament context, table dynamics,
    and the betting action that occurred before your turn.
    """
    description = "\n--- Poker Tournament Scenario ---\n"
    description += f"Tournament Stage: {scenario['tournament_stage']}\n"
    description += f"Current Street: {scenario['street']}\n"
    description += f"Your Hole Cards: {', '.join(scenario['your_hole_cards'])}\n"
    if scenario['street'] != 'Preflop':
        description += f"Board Cards: {', '.join(scenario['board'])}\n"
    description += f"Players in Hand (including you): {scenario['players_in_hand']}\n"
    description += f"Your Position: {scenario['your_position']}\n"
    description += f"Your Chip Stack: {scenario['your_chip_stack']}\n"
    description += f"Pot Size: {scenario['pot_size']}\n"
    description += "\nActions before you:\n"
    for idx, action in enumerate(scenario['opponent_actions'], 1):
        description += f"  Opponent {idx}: {action}\n"
    description += "\nAvailable Options for Your Action: " + ", ".join(scenario['available_options']) + "\n"
    return description

def run_scenario_generator(num_scenarios=5):
    """
    Generates a specified number of scenarios and prints the details for each scenario.
    """
    for i in range(num_scenarios):
        scenario = generate_scenario()
        print(f"\n=== Scenario {i+1} ===")
        print(describe_scenario(scenario))

# ---------------------------
# Main Program
# ---------------------------

if __name__ == '__main__':
    print("Poker AI Pro Trainer Scenario Generator (No Decision Input)")
    try:
        num = int(input("How many scenarios would you like to generate? (default 5): ").strip() or 5)
    except ValueError:
        num = 5
    run_scenario_generator(num)


Poker AI Pro Trainer Scenario Generator (No Decision Input)
How many scenarios would you like to generate? (default 5): 5

=== Scenario 1 ===

--- Poker Tournament Scenario ---
Tournament Stage: Late
Current Street: River
Your Hole Cards: J♣, K♣
Board Cards: 7♥, 7♦, 2♠, 2♦, K♥
Players in Hand (including you): 2
Your Position: Cutoff
Your Chip Stack: Medium
Pot Size: Moderate

Actions before you:
  Opponent 1: All-in

Available Options for Your Action: Check, Call, Raise, Fold, Re-raise All-in


=== Scenario 2 ===

--- Poker Tournament Scenario ---
Tournament Stage: Bubble
Current Street: River
Your Hole Cards: 7♠, A♦
Board Cards: 3♣, K♠, 8♣, Q♦, 6♠
Players in Hand (including you): 7
Your Position: Middle Position
Your Chip Stack: Deep
Pot Size: Small

Actions before you:
  Opponent 1: Raise 3x Blind
  Opponent 2: Call
  Opponent 3: All-in
  Opponent 4: All-in

Available Options for Your Action: Check, Call, Raise, Fold, Re-raise All-in


=== Scenario 3 ===

--- Poker Tournament Scenari